In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install torchlibrosa

In [ ]:
def path_folder(x):
    folder1 = "ab"
    folder2 = "cdef"
    folder3 = "ghijklm"
    folder4 = "nopqr"
    folder5 = "stuvwxy"
    if x[0] in folder1:
        return "/kaggle/input/birdsong-resampled-train-audio-00"
    elif x[0] in folder2:
        return "/kaggle/input/birdsong-resampled-train-audio-01"
    elif x[0] in folder3:
        return "/kaggle/input/birdsong-resampled-train-audio-02"
    elif x[0] in folder4:
        return "/kaggle/input/birdsong-resampled-train-audio-03"
    elif x[0] in folder5:
        return "/kaggle/input/birdsong-resampled-train-audio-04"
    else:
        raise("ERROR")
def path_folder_ext_wav(x):
    folder1 = "ab"
    folder2 = "cdef"
    folder3 = "ghijklm"
    folder4= "nopqrs"
    folder5 = "tuvwxyz"
    if x[0] in folder1:
        return "/kaggle/input/xenoexternalwav0/external-xeno-wav-0/external-xeno-wav"
    elif x[0] in folder2:
        return "/kaggle/input/xenoexternalwav0/external-xeno-wav-1/external-xeno-wav"
    elif x[0] in folder3:
        return "/kaggle/input/xenoexternalwav0/external-xeno-wav-2/external-xeno-wav"    
    elif x[0] in folder4:
        return "/kaggle/input/xenoexternalwav1/external-xeno-wav-3/external-xeno-wav" 
    elif x[0] in folder5:
        return "/kaggle/input/xenoexternalwav1/external-xeno-wav-4/external-xeno-wav" 
    else:
        print(x[0])
        raise("Error")
# Options for Logmel
mel_bins = 64
fmin = 50
fmax = 14000
window_size = 1024
hop_size = 320
audioset_classes_num = 527

pretrained_model_path = "pretrained/Cnn14_mAP0.431.pth"



data = pd.read_csv("/kaggle/input/birdsongdataset/trainv2.csv").sort_values("filename").reset_index().drop(columns=["index"])
data["path"] = data.apply(lambda x: os.path.join(path_folder(x.ebird_code), x.ebird_code, x.filename.replace(".mp3", ".wav")), axis=1)
data["targets2"] = (~data.background.isna()).astype(np.float32)

data_ext = pd.read_csv("/kaggle/input/birdsongdataset/trainv2_ext.csv").sort_values("filename").reset_index().drop(columns=["index"])
data_ext["path"] = data_ext.apply(lambda x: os.path.join(path_folder_ext_wav(x.ebird_code), x.ebird_code, x.filename.replace(".mp3", ".wav")), axis=1)
data_ext["targets2"] = (~data_ext.background.isna()).astype(np.float32)

train = data.query("fold != 0").reset_index().drop(columns=["index"])
val = data.query("fold == 0").reset_index().drop(columns=["index"])

train = pd.concat((train, data_ext), axis=0).reset_index().drop(columns=["index"])

val_5sec = pd.read_csv("/kaggle/input/birdsongdataset/validation_5sec.csv")
val_5sec["path"] = val_5sec.apply(lambda x: os.path.join(path_folder(x.ebird_code) if x.split == "train" else path_folder_ext_wav(x.ebird_code), x.ebird_code, x.filename.replace(".mp3", ".wav")), axis=1)
val_5sec["targets2"] = (~val_5sec.background.isna()).astype(np.float32)
noise = pd.read_csv("/kaggle/input/esc50dataset/esc_environment.csv")
noise["path"] = noise.apply(lambda x: os.path.join("/kaggle/input/esc50dataset/ESC-50-master/ESC-50-master/audio", x.filename), axis=1)

In [ ]:
train

In [ ]:
val

In [ ]:
val_5sec

In [ ]:
noise

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler 
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation
from torch.nn.utils.rnn import *
import random

In [ ]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = False
        
seed_everything(42)

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler 

import torchvision
import torchvision.models as models

import librosa
import librosa.display

from joblib import Parallel, delayed
import os
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio, IFrame, display

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm

from sklearn.metrics import f1_score
import pickle
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [ ]:
class ConvPreWavBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvPreWavBlock, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=3, stride=1,
                              padding=1, bias=False)
                              
        self.conv2 = nn.Conv1d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=3, stride=1, dilation=2, 
                              padding=2, bias=False)
                              
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        x = F.max_pool1d(x, kernel_size=pool_size)
        
        return x


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x


def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)
    
    
class Wavegram_Logmel_Cnn14(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, 
        fmax, classes_num):
        
        super(Wavegram_Logmel_Cnn14, self).__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.classes_num = classes_num
        # Wavegram
        self.pre_conv0 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=11, stride=5, padding=5, bias=False)
        self.pre_bn0 = nn.BatchNorm1d(64)
        self.pre_block1 = ConvPreWavBlock(64, 64)
        self.pre_block2 = ConvPreWavBlock(64, 128)
        self.pre_block3 = ConvPreWavBlock(128, 128)
        self.pre_block4 = ConvBlock(in_channels=4, out_channels=64)

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=128, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.fc_audioset = nn.Linear(2048, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_layer(self.pre_conv0)
        init_bn(self.pre_bn0)
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)
 
    def forward(self, input, mixup_lambda=None):
        """ Input: (batch_size, data_length)"""

        a1 = F.relu_(self.pre_bn0(self.pre_conv0(input[:, None, :])))
        a1 = self.pre_block1(a1, pool_size=4)
        a1 = self.pre_block2(a1, pool_size=4)
        a1 = self.pre_block3(a1, pool_size=4)
        a1 = a1.reshape((a1.shape[0], -1, 32, a1.shape[-1])).transpose(2, 3)
        a1 = self.pre_block4(a1, pool_size=(2, 1))
        #print("a1:", a1)
        x1 = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        #print("spect", x.isnan().sum(), x.shape, x[0])
        with autocast(False):
            x = self.logmel_extractor(x1)    # (batch_size, 1, time_steps, mel_bins)
        #if x.isnan().sum()>0:
            #print(x1[0,0,0], x[0,0,0])
            #print("logmel", x.isnan().sum(), x.shape)
            #x[x.isnan()] = 0.
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
            a1 = do_mixup(a1, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')

        # Concate wavegram and spectrogram along the channel dimention
        x = torch.cat((x, a1), dim=1)


        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        
        #embedding = F.dropout(x, p=0.5, training=self.training)
        clipwise_output = self.fc_audioset(x)
        #print("clipwise:",clipwise_output)
        #output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return torch.sigmoid(clipwise_output)#output_dict 



In [ ]:
def _resnet_conv3x3(in_planes, out_planes):
    #3x3 convolution with padding
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1,
                     padding=1, groups=1, bias=False, dilation=1)


def _resnet_conv1x1(in_planes, out_planes):
    #1x1 convolution
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, bias=False)


class _ResnetBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(_ResnetBasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('_ResnetBasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in _ResnetBasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1

        self.stride = stride

        self.conv1 = _resnet_conv3x3(inplanes, planes)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = _resnet_conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

        self.init_weights()

    def init_weights(self):
        init_layer(self.conv1)
        init_bn(self.bn1)
        init_layer(self.conv2)
        init_bn(self.bn2)
        nn.init.constant_(self.bn2.weight, 0)

    def forward(self, x):
        identity = x

        if self.stride == 2:
            out = F.avg_pool2d(x, kernel_size=(2, 2))
        else:
            out = x

        out = self.conv1(out)
        out = self.bn1(out)
        out = self.relu(out)
        out = F.dropout(out, p=0.1, training=self.training)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)

        return out


class _ResnetBottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(_ResnetBottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        self.stride = stride
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = _resnet_conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = _resnet_conv3x3(width, width)
        self.bn2 = norm_layer(width)
        self.conv3 = _resnet_conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

        self.init_weights()

    def init_weights(self):
        init_layer(self.conv1)
        init_bn(self.bn1)
        init_layer(self.conv2)
        init_bn(self.bn2)
        init_layer(self.conv3)
        init_bn(self.bn3)
        nn.init.constant_(self.bn3.weight, 0)

    def forward(self, x):
        identity = x

        if self.stride == 2:
            x = F.avg_pool2d(x, kernel_size=(2, 2))

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = F.dropout(out, p=0.1, training=self.training)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)

        return out


class _ResNet(nn.Module):
    def __init__(self, block, layers, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(_ResNet, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group

        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            if stride == 1:
                downsample = nn.Sequential(
                    _resnet_conv1x1(self.inplanes, planes * block.expansion),
                    norm_layer(planes * block.expansion),
                )
                init_layer(downsample[0])
                init_bn(downsample[1])
            elif stride == 2:
                downsample = nn.Sequential(
                    nn.AvgPool2d(kernel_size=2), 
                    _resnet_conv1x1(self.inplanes, planes * block.expansion),
                    norm_layer(planes * block.expansion),
                )
                init_layer(downsample[1])
                init_bn(downsample[2])

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        return x

In [ ]:
class ResNet22(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, 
        fmax, classes_num):
        
        super(ResNet22, self).__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.classes_num = classes_num
        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        # self.conv_block2 = ConvBlock(in_channels=64, out_channels=64)

        self.resnet = _ResNet(block=_ResnetBasicBlock, layers=[2, 2, 2, 2], zero_init_residual=True)

        self.conv_block_after1 = ConvBlock(in_channels=512, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048)
        self.fc_audioset = nn.Linear(2048, classes_num, bias=True)

        self.init_weights()

    def init_weights(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)


    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        with autocast(False):
            x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = self.resnet(x)
        x = F.avg_pool2d(x, kernel_size=(2, 2))
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = self.conv_block_after1(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        #embedding = F.dropout(x, p=0.5, training=self.training)
        clipwise_output = torch.sigmoid(self.fc_audioset(x))
        
        #output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return clipwise_output#output_dict


class ResNet38(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, fmax, classes_num, background=False):
        
        super(ResNet38, self).__init__()
        self.classes_num = classes_num
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        # self.conv_block2 = ConvBlock(in_channels=64, out_channels=64)

        self.resnet = _ResNet(block=_ResnetBasicBlock, layers=[3, 4, 6, 3], zero_init_residual=True)

        self.conv_block_after1 = ConvBlock(in_channels=512, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048)
        self.fc_audioset = nn.Linear(2048, classes_num, bias=True)
        
        self.background = background
        if background:
            self.fc_background = nn.Linear(2048, 1, bias=True)
        self.init_weights()

    def init_weights(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)


    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        with autocast(False):
            x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = self.resnet(x)
        x = F.avg_pool2d(x, kernel_size=(2, 2))
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = self.conv_block_after1(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        #embedding = F.dropout(x, p=0.5, training=self.training)
        clipwise_output = torch.sigmoid(self.fc_audioset(x))
        
        if self.background:
            background_output = torch.sigmoid(self.fc_background(x))
            return clipwise_output, background_output#output_dict
        return clipwise_output

class ResNet54(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, fmax, classes_num):
        
        super(ResNet54, self).__init__()
        self.classes_num = classes_num
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        # self.conv_block2 = ConvBlock(in_channels=64, out_channels=64)

        self.resnet = _ResNet(block=_ResnetBottleneck, layers=[3, 4, 6, 3], zero_init_residual=True)

        self.conv_block_after1 = ConvBlock(in_channels=2048, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048)
        self.fc_audioset = nn.Linear(2048, classes_num, bias=True)

        self.init_weights()

    def init_weights(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)


    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        with autocast(False):
            x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = self.resnet(x)
        x = F.avg_pool2d(x, kernel_size=(2, 2))
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = self.conv_block_after1(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        #embedding = F.dropout(x, p=0.5, training=self.training)
        clipwise_output = torch.sigmoid(self.fc_audioset(x))
        
        #output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return clipwise_output #output_dict


In [ ]:
class GlobalAttention(nn.Module):
    def __init__(self, hidden_size, padding_value=0):
        super(GlobalAttention , self).__init__()
        
        self.W = nn.Linear(hidden_size, hidden_size)#, bias=False)
        self.padding_value = padding_value
    def forward(self, h_i_pack):
        """
            input is a pack seq format
        """
        
        h_i, _ = pad_packed_sequence(h_i_pack, batch_first=True, padding_value=self.padding_value)
        #print("pad seq", h_i.shape)
        #print(self.W.weight.shape, h_i_pack[0].shape)
        u_i = self.W(h_i_pack[0])
        #print("u_i :", u_i.isnan().sum(), u_i.shape, u_i)
        #print("compute w")
        u_i = PackedSequence(u_i, h_i_pack[1])
        
        u_i, _ = pad_packed_sequence(u_i, batch_first=True, padding_value=-float("inf"))
        #print("pad soft")
        a_i =  F.softmax(u_i, dim=1)#µ/2.0
        #if a_i.isnan().sum()>0:
        #    print("Attention globale : ", a_i.shape, a_i)
        
        #print(a_i.shape, a_i.sum(1), a_i.sum(1).shape)
        v = (a_i * h_i).sum(1)        
        #print(v)
        return v, a_i

class BirdSongClassifierBackground(nn.Module):
    def __init__(self, backbone, freeze_base=False, background=False):
        """Classifier for a new task using pretrained Cnn14 as a sub module.
        """
        super(BirdSongClassifierBackground, self).__init__()        
        self.backbone = backbone

        self.global_attention = GlobalAttention(backbone.classes_num)
        self.background = background
        if background:
            self.global_attention_background = GlobalAttention(1)

        if freeze_base:
            # Freeze AudioSet pretrained layers
            for param in self.backbone.parameters():
                param.requires_grad = False

   

 

    def load_from_pretrain(self, pretrained_checkpoint_path):
        checkpoint = torch.load(pretrained_checkpoint_path,
                                map_location="cpu")
        
        try:
            del checkpoint["model"]["fc_audioset.weight"]
            del checkpoint["model"]["fc_audioset.bias"]
            print("checkpoint audioset deleted")
        except:
            try:
                del checkpoint["model"]["att_block.att.weight"]
                del checkpoint["model"]["att_block.att.bias"]
                del checkpoint["model"]["att_block.cla.weight"]
                del checkpoint["model"]["att_block.cla.bias"]
                del checkpoint["model"]["att_block.bn_att.weight"]
                del checkpoint["model"]["att_block.bn_att.bias"]
                del checkpoint["model"]["att_block.bn_att.running_mean"]
                del checkpoint["model"]["att_block.bn_att.running_var"]
                print("checkpoint attention deleted")
            except:
                pass
        weights = self.backbone.state_dict()
        weights.update(checkpoint['model'])
        self.backbone.load_state_dict(weights)

    def forward(self, input, mixup_lambda=None, lengths=None):
        """Input: (batch_size, length, data_length)
        """

        if self.background:
            features = []
            features2 = []
            for i in range(input.shape[1]):
                preds_chunk, preds_chunk2 = self.backbone(input[:, i], mixup_lambda)
                #print("nan : ",preds_chunk.isnan().sum())
                features.append(preds_chunk)
                features2.append(preds_chunk2)

            features = torch.stack(features, dim=1) # prediction per chunk of 5 sec
            features2 = torch.stack(features2, dim=1) # prediction per chunk of 5 sec
            #print(len(lengths), features.shape)

            if mixup_lambda is not None:
                lengths = do_mixup_length(lengths)
            features_pack = pack_padded_sequence(features, batch_first=True, lengths=lengths)
            features2_pack = pack_padded_sequence(features2, batch_first=True, lengths=lengths)

            x , att = self.global_attention(features_pack)
            x2 , att2 = self.global_attention_background(features2_pack)

            return x , features , x2, features2
        
        else:
            features = []
        
            for i in range(input.shape[1]):
                preds_chunk = self.backbone(input[:, i], mixup_lambda)
                features.append(preds_chunk)

            features = torch.stack(features, dim=1) # prediction per chunk of 5 sec

            if mixup_lambda is not None:
                lengths = do_mixup_length(lengths)
            features_pack = pack_padded_sequence(features, batch_first=True, lengths=lengths)

            x , att = self.global_attention(features_pack)

            return x , features


In [ ]:
class AttBlock(nn.Module):
    def __init__(self, n_in, n_out, activation='linear', temperature=1.):
        super(AttBlock, self).__init__()
        
        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(in_channels=n_in, out_channels=n_out, kernel_size=1, stride=1, padding=0, bias=True)
        self.cla = nn.Conv1d(in_channels=n_in, out_channels=n_out, kernel_size=1, stride=1, padding=0, bias=True)
        
        self.bn_att = nn.BatchNorm1d(n_out)
        self.init_weights()
        
    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)
         
    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

class Cnn14_DecisionLevelAtt(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, 
        fmax, classes_num, background=False):
        
        super(Cnn14_DecisionLevelAtt, self).__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.classes_num = classes_num
        self.interpolate_ratio = 32     # Downsampled ratio

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.att_block = AttBlock(2048, self.classes_num, activation='sigmoid')
        self.background = background
        if background:
            self.att_block2 = AttBlock(2048, 1, activation='sigmoid')

        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
 
    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        with autocast(False):
            x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)

        frames_num = x.shape[2]
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.mean(x, dim=3)
        
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, _, segmentwise_output) = self.att_block(x)

        (clipwise_output2, _, segmentwise_output2) = self.att_block2(x)

        #segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        #framewise_output = interpolate(segmentwise_output, self.interpolate_ratio)
        #framewise_output = pad_framewise_output(framewise_output, frames_num)

        #output_dict = {'framewise_output': framewise_output, 
        #    'clipwise_output': clipwise_output}
        if self.background:

            return clipwise_output, clipwise_output2
        return clipwise_output #output_dict

In [ ]:
class AudioGeneratorDataset(torch.utils.data.Dataset):
    def __init__(self, path_audio, y, y2, path_augment,  resample_freq = 32000, 
                 max_length=365, augmentation=[], validation=False, num_class=264, background=False):
        self.labels2idx = {'aldfly': 0, 'ameavo': 1, 'amebit': 2, 'amecro': 3, 'amegfi': 4,
    'amekes': 5, 'amepip': 6, 'amered': 7, 'amerob': 8, 'amewig': 9,
    'amewoo': 10, 'amtspa': 11, 'annhum': 12, 'astfly': 13, 'baisan': 14,
    'baleag': 15, 'balori': 16, 'banswa': 17, 'barswa': 18, 'bawwar': 19,
    'belkin1': 20, 'belspa2': 21, 'bewwre': 22, 'bkbcuc': 23, 'bkbmag1': 24,
    'bkbwar': 25, 'bkcchi': 26, 'bkchum': 27, 'bkhgro': 28, 'bkpwar': 29,
    'bktspa': 30, 'blkpho': 31, 'blugrb1': 32, 'blujay': 33, 'bnhcow': 34,
    'boboli': 35, 'bongul': 36, 'brdowl': 37, 'brebla': 38, 'brespa': 39,
    'brncre': 40, 'brnthr': 41, 'brthum': 42, 'brwhaw': 43, 'btbwar': 44,
    'btnwar': 45, 'btywar': 46, 'buffle': 47, 'buggna': 48, 'buhvir': 49,
    'bulori': 50, 'bushti': 51, 'buwtea': 52, 'buwwar': 53, 'cacwre': 54,
    'calgul': 55, 'calqua': 56, 'camwar': 57, 'cangoo': 58, 'canwar': 59,
    'canwre': 60, 'carwre': 61, 'casfin': 62, 'caster1': 63, 'casvir': 64,
    'cedwax': 65, 'chispa': 66, 'chiswi': 67, 'chswar': 68, 'chukar': 69,
    'clanut': 70, 'cliswa': 71, 'comgol': 72, 'comgra': 73, 'comloo': 74,
    'commer': 75, 'comnig': 76, 'comrav': 77, 'comred': 78, 'comter': 79,
    'comyel': 80, 'coohaw': 81, 'coshum': 82, 'cowscj1': 83, 'daejun': 84,
    'doccor': 85, 'dowwoo': 86, 'dusfly': 87, 'eargre': 88, 'easblu': 89,
    'easkin': 90, 'easmea': 91, 'easpho': 92, 'eastow': 93, 'eawpew': 94,
    'eucdov': 95, 'eursta': 96, 'evegro': 97, 'fiespa': 98, 'fiscro': 99,
    'foxspa': 100, 'gadwal': 101, 'gcrfin': 102, 'gnttow': 103, 'gnwtea': 104,
    'gockin': 105, 'gocspa': 106, 'goleag': 107, 'grbher3': 108, 'grcfly': 109,
    'greegr': 110, 'greroa': 111, 'greyel': 112, 'grhowl': 113, 'grnher': 114,
    'grtgra': 115, 'grycat': 116, 'gryfly': 117, 'haiwoo': 118, 'hamfly': 119,
    'hergul': 120, 'herthr': 121, 'hoomer': 122, 'hoowar': 123, 'horgre': 124,
    'horlar': 125, 'houfin': 126, 'houspa': 127, 'houwre': 128, 'indbun': 129,
    'juntit1': 130, 'killde': 131, 'labwoo': 132, 'larspa': 133, 'lazbun': 134,
    'leabit': 135, 'leafly': 136, 'leasan': 137, 'lecthr': 138, 'lesgol': 139,
    'lesnig': 140, 'lesyel': 141, 'lewwoo': 142, 'linspa': 143, 'lobcur': 144,
    'lobdow': 145, 'logshr': 146, 'lotduc': 147, 'louwat': 148, 'macwar': 149,
    'magwar': 150, 'mallar3': 151, 'marwre': 152, 'merlin': 153, 'moublu': 154,
    'mouchi': 155, 'moudov': 156, 'norcar': 157, 'norfli': 158, 'norhar2': 159,
    'normoc': 160, 'norpar': 161, 'norpin': 162, 'norsho': 163, 'norwat': 164,
    'nrwswa': 165, 'nutwoo': 166, 'olsfly': 167, 'orcwar': 168, 'osprey': 169,
    'ovenbi1': 170, 'palwar': 171, 'pasfly': 172, 'pecsan': 173, 'perfal': 174,
    'phaino': 175, 'pibgre': 176, 'pilwoo': 177, 'pingro': 178, 'pinjay': 179,
    'pinsis': 180, 'pinwar': 181, 'plsvir': 182, 'prawar': 183, 'purfin': 184,
    'pygnut': 185, 'rebmer': 186, 'rebnut': 187, 'rebsap': 188, 'rebwoo': 189,
    'redcro': 190, 'redhea': 191, 'reevir1': 192, 'renpha': 193, 'reshaw': 194,
    'rethaw': 195, 'rewbla': 196, 'ribgul': 197, 'rinduc': 198, 'robgro': 199,
    'rocpig': 200, 'rocwre': 201, 'rthhum': 202, 'ruckin': 203, 'rudduc': 204,
    'rufgro': 205, 'rufhum': 206, 'rusbla': 207, 'sagspa1': 208, 'sagthr': 209,
    'savspa': 210, 'saypho': 211, 'scatan': 212, 'scoori': 213, 'semplo': 214,
    'semsan': 215, 'sheowl': 216, 'shshaw': 217, 'snobun': 218, 'snogoo': 219,
    'solsan': 220, 'sonspa': 221, 'sora': 222, 'sposan': 223, 'spotow': 224,
    'stejay': 225, 'swahaw': 226, 'swaspa': 227, 'swathr': 228, 'treswa': 229,
    'truswa': 230, 'tuftit': 231, 'tunswa': 232, 'veery': 233, 'vesspa': 234,
    'vigswa': 235, 'warvir': 236, 'wesblu': 237, 'wesgre': 238, 'weskin': 239,
    'wesmea': 240, 'wessan': 241, 'westan': 242, 'wewpew': 243, 'whbnut': 244,
    'whcspa': 245, 'whfibi': 246, 'whtspa': 247, 'whtswi': 248, 'wilfly': 249,
    'wilsni1': 250, 'wiltur': 251, 'winwre3': 252, 'wlswar': 253, 'wooduc': 254,
    'wooscj2': 255, 'woothr': 256, 'y00475': 257, 'yebfly': 258, 'yebsap': 259,
    'yehbla': 260, 'yelwar': 261, 'yerwar': 262, 'yetvir': 263}
        
        self.idx2labels = {k:v for v,k in self.labels2idx.items()}

        self.augmentation = set(augmentation)
        self.samples = path_audio #+ path_augment
        self.max_length = max_length # 99% are shorter than 365 sec
        self.resample_freq=resample_freq
        self.validation = validation
        self.y = np.array([self.labels2idx[t] for t in y]) #+ [self.labels2idx[t] for t in y_aug]
        self.background = background
        if self.background:
            self.y2 = np.array(y2)
        self.num_class = num_class
        if "noise" in self.augmentation:
            self.noise = [self.load_raw_audio_noise(p) for p in path_augment]

    def load_raw_audio_noise(self, x):
        signal, sr_orig = librosa.load(x, sr=self.resample_freq)
        shape = len(signal)
        
        if shape > self.max_length*self.resample_freq:
            start = np.random.randint(0, shape - self.max_length*self.resample_freq)
            signal = signal[start:start+self.max_length*self.resample_freq]
            if len(signal) != self.max_length*self.resample_freq:
                signal = np.resize(signal, self.max_length*self.resample_freq).astype(np.float32)
        else:
            toadd = (self.resample_freq*5 -  (signal.shape[0]%(self.resample_freq*5)))%(self.resample_freq*5)
            signal = np.resize(signal, self.max_length*self.resample_freq).astype(np.float32)
        return signal
            
    def load_raw_audio(self, x):
        signal, sr_orig = librosa.load(x, sr=self.resample_freq)
        shape = len(signal)
        if self.validation:
                signal = signal[:self.max_length*self.resample_freq]
        else:
            if shape > self.max_length*self.resample_freq:
                start = np.random.randint(0, shape - self.max_length*self.resample_freq)
                signal = signal[start:start+self.max_length*self.resample_freq]
                #print(len(signal))
        toadd = (self.resample_freq*5 -  (signal.shape[0]%(self.resample_freq*5)))%(self.resample_freq*5)
        signal = np.concatenate((signal, np.zeros(toadd))).astype(np.float32)
        return signal.astype(np.float32)
    
    def load_pickle(self, filename):
        #with open(filename, "rb") as f:
        data = np.load(filename).astype(np.float32)#pickle.load(f)
        return data
    

    def load_audio(self, filename):

            signal = self.load_pickle(filename)
            shape = len(signal)
            #print("shape:", shape)
            if self.validation:
                signal = signal[:self.max_length*self.resample_freq]
            else:
                if shape > self.max_length*self.resample_freq:
                    start = np.random.randint(0, shape - self.max_length*self.resample_freq)
                    signal = signal[start:start+self.max_length*self.resample_freq]
                    #print(len(signal))
            toadd = (self.resample_freq*5 -  (signal.shape[0]%(self.resample_freq*5)))%(self.resample_freq*5)
            signal = np.concatenate((signal, np.zeros(toadd))).astype(np.float32)
            return signal
    def normalise(self, x):
        """
        

        Parameters
        ----------
        x : numpy array
            DESCRIPTION.

        Returns
        -------
        x_norm : Normalise signal between 0 and 1 value
            DESCRIPTION.

        """
        min_ = x.min()
        max_ = x.max()
        if min_ == max_:
            x_norm = np.zeros(x.shape).astype(np.float32)
        else:
            x_norm =  (x-min_)/(max_ - min_)
        return x_norm
    
    def __getitem__(self, index):
        l = []
        # label
        labels_item = set([self.y[index]])
        labels_one_hot = torch.nn.functional.one_hot(torch.tensor(self.y[index]), self.num_class).type(torch.float32)
        if self.background:
            labels2_item = self.y2[index]
        # load signal
        signal_raw = self.load_raw_audio(self.samples[index] )
        # add multiple audio (Augmentation for multi labels)
        if "augment" in self.augmentation:
            augmented = np.zeros(signal_raw.shape)
            number = np.random.randint(0, 3)
            ids = np.random.randint(0, self.__len__(), size=number)
            counts = 0
            for id_ in ids:
                if self.y[id_] not in labels_item: # add only if class not already present
                    labels_item.add(self.y[id_])
                    labels_one_hot += torch.nn.functional.one_hot(torch.tensor(self.y[id_]), self.num_class).type(torch.float32) # update label
                    augmented += np.resize(self.load_raw_audio(self.samples[id_] ), augmented.shape) # update augmented signal
                    if self.background:
                        labels2_item += self.y2[id_]
                    counts += 1
            if counts > 0:
                counts += 1
                signal_raw = (signal_raw + augmented)/counts
        # add Environment Noise
        
        if "noise" and "gaussian" in self.augmentation:
            if np.random.uniform()>0.5:
                noise = np.resize(self.noise[np.random.randint(0, len(self.noise))], len(signal_raw))
                alpha = np.random.uniform(0.1,0.3)
                signal_raw = (alpha*signal_raw) + ((1-alpha)*noise)
                
            else:
                signal_raw = AddGaussianNoise(signal_raw)
        else:
            if "noise" in self.augmentation:
                noise = np.resize(self.noise[np.random.randint(0, len(self.noise))], len(signal_raw))
                alpha = np.random.uniform(0.1,0.3)
                signal_raw = (alpha*signal_raw) + ((1-alpha)*noise)
                
            if "gaussian" in self.augmentation:
                signal_raw = AddGaussianNoise(signal_raw)
        ## convert into mel spectrogram
        signal_raw = signal_raw.reshape(-1, self.resample_freq*5)
        
        
        l.append( torch.tensor(signal_raw) )
        l.append(torch.tensor(len(signal_raw)))
        l.append(labels_one_hot)
        
        if self.background:
            labels2_item = torch.tensor(labels2_item>0).float()
            l.append(labels2_item)
        l.append(torch.tensor(index))
        return tuple(l)
    
    def __len__(self):
        return len(self.samples)
    
    
def collate_padding_background(batch):
    lengths = []
    signals = []
    indices = []
    labels = []
    labels2 = []
    max_ = 0
    for item in batch :
        signals.append(item[0])
        lengths.append(item[1])
        max_ = max(max_, item[1])
        labels.append(item[2])
        labels2.append(item[3])
        indices.append(item[4])
    
    #print(lengths)
    lengths = torch.stack(lengths)
    indices = torch.stack(indices)
    labels = torch.stack(labels)
    labels2 = torch.stack(labels2)

    for i in range(len(lengths)):
        N = max_ - len(signals[i])
        if N > 0:
            signals[i] = torch.cat((signals[i], torch.zeros((N, signals[i].shape[1]))), dim=0)
    signals = torch.stack(signals, dim=0)
    
    idx_sort = torch.argsort(lengths, descending=True)
    
        
    lengths = lengths[idx_sort]
    labels = labels[idx_sort]
    labels2 = labels2[idx_sort]

    indices = indices[idx_sort]
    signals = signals[idx_sort]
    
    return signals, lengths, labels, labels2, indices

def collate_padding(batch):
    lengths = []
    signals = []
    indices = []
    labels = []
    max_ = 0
    for item in batch :
        signals.append(item[0])
        lengths.append(item[1])
        max_ = max(max_, item[1])
        labels.append(item[2])
        indices.append(item[-1])
    
    #print(lengths)
    lengths = torch.stack(lengths)
    indices = torch.stack(indices)
    labels = torch.stack(labels)

    for i in range(len(lengths)):
        N = max_ - len(signals[i])
        if N > 0:
            signals[i] = torch.cat((signals[i], torch.zeros((N, signals[i].shape[1]))), dim=0)
    signals = torch.stack(signals, dim=0)
    
    idx_sort = torch.argsort(lengths, descending=True)
    
        
    lengths = lengths[idx_sort]
    labels = labels[idx_sort]
    
    indices = indices[idx_sort]
    signals = signals[idx_sort]
    
    return signals, lengths, labels, indices    

In [ ]:
class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return np.array(mixup_lambdas)

In [ ]:
## -- Fonction Loop  -- ##
    

        
def train_fn(model, dataloader, optimizer, loss_fn, cfg, accumulation=2, l_mixup=1.0):
    total_loss = 0.
    model.train()
    t=tqdm(dataloader)
    scaler = GradScaler()
    optimizer.zero_grad()
    N = 0.
    if l_mixup>0:
        mixup = Mixup(mixup_alpha=l_mixup)

    for i, batch in enumerate(t):
        inputs= batch[0]
        lengths= batch[1]
        labels= batch[2]
        if cfg.background:
            labels2= batch[3]
            labels2 = labels2.to(device, dtype=torch.float)
        indices = batch[-1]        
        
        inputs = inputs.to(cfg.device, dtype=torch.float)
        lengths = lengths.to(cfg.device, dtype=torch.float)
        labels = labels.to(cfg.device, dtype=torch.float)
 
        lambda_mixup = None
        if l_mixup>0:
            lambda_mixup = torch.as_tensor(mixup.get_lambda(batch_size=cfg.batch_size)).to(cfg.device, dtype=torch.float)
            labels = do_mixup(labels, lambda_mixup)
            if cfg.background:
                labels2 = do_mixup(labels2, lambda_mixup)
                labels2[labels2>1.0] = 1.0
            labels[labels>1.0] = 1.0
            
            
        with autocast(cfg.use_apex):
            if cfg.background:
                outputs, outputs_frames, outputs2, _ = model(inputs, lambda_mixup, lengths)
                outputs2 = torch.clamp(outputs2,0.0,1.0)
            else:
                outputs, outputs_frames = model(inputs, lambda_mixup, lengths)
            outputs = torch.clamp(outputs,0.0,1.0)
            labels = torch.clamp(labels,0.0,1.0)
            
            
            
            loss = loss_fn(outputs, labels )
            if cfg.background:
                loss2 = loss_fn(outputs2.view(-1) ,labels2)
                loss = (loss.sum(1)+loss2).mean()
            else:
                loss = loss.sum(1).mean()
            N += len(inputs)

            if torch.isnan(loss):
                print("loss error")
                print(torch.isnan(outputs).sum())
                loss[torch.isnan(loss)] = 0.0

                total_loss += (loss.item() * len(inputs))
            else:
                total_loss += (loss.item() * len(inputs))


        if cfg.use_apex:
            loss = loss/accumulation
            scaler.scale(loss).backward()
        else:
            loss = loss/accumulation
            loss.backward()

        



        if (i+1)%accumulation == 0 or i-1 == len(t):
            if cfg.use_apex:
                scaler.step(optimizer)
    
                # Updates the scale for next iteration.
                scaler.update()
                optimizer.zero_grad()
            else:                
                optimizer.step()
                optimizer.zero_grad()
            
        
        t.set_description("Loss : {0}".format(total_loss/N))
        t.refresh()       

    return total_loss/N
        
def inference_fn(model, dataloader, optimizer, device, loss_fn, activation=False, background=False):
    total_loss = 0.
    t=tqdm(dataloader)
    y_true = []
    y_preds = []
    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(t):
            
            inputs= batch[0]
            lengths= batch[1]
            labels= batch[2]
            if background:
                labels2= batch[3]
                labels2 = labels2.to(device, dtype=torch.float)
            indices = batch[-1]

            inputs = inputs.to(device, dtype=torch.float)
            lengths = lengths.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)
            

            #labels_one_hot = torch.cuda.FloatTensor(labels.size(0), 265).to(device, dtype=torch.float)
            #labels_one_hot = torch.nn.functional.one_hot(labels, 264).to(device, dtype=torch.float)
            with autocast(cfg.use_apex):
                if background:
                    outputs, outputs_frames, outputs2, _ = model(inputs, None, lengths)
                    outputs2 = torch.clamp(outputs2,0.0,1.0)
                    labels2 = torch.clamp(labels2,0.0,1.0)
                else:
                    outputs, outputs_frames  = model(inputs, None, lengths)
                    
                outputs = torch.clamp(outputs,0.0,1.0)
                labels = torch.clamp(labels,0.0,1.0)
                                    

                loss = loss_fn(outputs, labels )

                loss = loss.sum(1).mean()
            
                total_loss += loss.item()
            
            t.set_description("Loss : {0}".format(total_loss/(i+1)))
            t.refresh()
        
            y_true.append(labels.detach().cpu().numpy())
            if activation:
                outputs = torch.sigmoid(outputs)
            y_preds.append(outputs.cpu().detach().numpy())
            
    return np.concatenate(y_preds), np.concatenate(y_true), total_loss/(i+1)

In [ ]:
def do_mixup(x, mixup_lambda):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes 
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0 :: 2].transpose(0, -1) * mixup_lambda[0 :: 2] + \
        x[1 :: 2].transpose(0, -1) * mixup_lambda[1 :: 2]).transpose(0, -1)
    return out

In [ ]:
def do_mixup_length(x):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes 
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = torch.stack( (x[0 :: 2] , x[1 :: 2]) ).max(0)[0]
    return out

do_mixup_length(torch.as_tensor([5,2,4,5,7,9,12,15,17,2]))

In [ ]:
## -- FONCTION DE COUTS -- ##
    
def bce(outputs, targets):
    eps = 1e-5
    p1=targets*(torch.log(outputs+eps))
    p0=(1-targets)*torch.log(1-outputs+eps)
    
    loss = p0 + p1
    return -loss
def f1_score_multi(y_true, y_preds, threshold=0.5):
    y_preds = (y_preds>=threshold).astype(np.int32)
    f1 = []
    for i in range(y_preds.shape[1]):
        if len(np.unique(y_true[:,i])) == 2:
            f1.append(f1_score(y_true[:,i], y_preds[:,i]))
    
    return f1

def f1_micro_row_wise(y_true, y_preds, threshold=0.5):
    y_preds = (y_preds>=threshold).astype(np.int32)
    return f1_score(y_true, y_preds, average='samples')
class Config():
    def __init__(self):
        self.num_class = 264
        self.resample_freq=32000
        self.max_length=30 # seconds
        self.device = "cuda:0"
        self.use_apex = False
        self.epochs = 6
        self.accumulation = 2
        self.batch_size = 16
        self.background = False
        self.l_mixup=0.0
        self.load_name = "../input/pretrained-pann/ResNet38_mAP0.434.pth"
        self.save_name = "ResNet38-264-birds-gaussian-ext.pth"

In [ ]:
cfg = Config()
cfg.batch_size = cfg.batch_size * 2 if cfg.l_mixup > 0. else cfg.batch_size

In [ ]:
def AddGaussianNoise(sound):
        noise = 0.005*np.random.uniform()*np.amax(sound)
        augmented_sound = np.array(sound).astype('float32') + noise * np.random.normal(size=sound.shape[0])
        return augmented_sound

In [ ]:
augmentations=["noise", "augment", "gaussian"]

In [ ]:
## -- DATASET / DATALOADER -- #

dataset = AudioGeneratorDataset(train.path.values.tolist(), train.ebird_code.values, train.targets2.values, noise.path.values.tolist(),
                                 resample_freq=cfg.resample_freq, max_length=cfg.max_length, 
                                augmentation=augmentations, validation=False, num_class=cfg.num_class, background=cfg.background)

dataset_val = AudioGeneratorDataset(val.path.values.tolist(), val.ebird_code.values, val.targets2.values, [],  
                                    resample_freq=cfg.resample_freq, max_length=cfg.max_length, augmentation=[], validation=True,
                                    num_class=cfg.num_class, background=cfg.background)


dataset_val5 = AudioGeneratorDataset(val_5sec.path.values.tolist(), val_5sec.ebird_code.values,val_5sec.targets2.values, [],  
                                    resample_freq=cfg.resample_freq, max_length=cfg.max_length, augmentation=[], validation=True,
                                    num_class=cfg.num_class, background=cfg.background)

loss_fnt = bce#nn.BCEWithLogitsLoss(reduction="none")
loss_fnt_test = nn.BCELoss(reduction="none")#nn.BCEWithLogitsLoss(reduction="none")

collate_fn = collate_padding_background if cfg.background else collate_padding
train_dataloader = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True, collate_fn=collate_fn, num_workers=4,drop_last=True)
val_dataloader = DataLoader(dataset_val, batch_size=cfg.batch_size, shuffle=False, collate_fn=collate_fn, num_workers=1)
val_dataloader5 = DataLoader(dataset_val5, batch_size=cfg.batch_size, shuffle=False, collate_fn=collate_fn, num_workers=1)





In [ ]:
## -- LOAD MODEL -- ##

#backbone =  Cnn14_DecisionLevelAtt(cfg.resample_freq, window_size, hop_size, mel_bins, fmin, fmax, cfg.num_class, background=True)

backbone = ResNet38(cfg.resample_freq, window_size, hop_size, mel_bins, fmin, fmax, cfg.num_class, background=cfg.background)

model = BirdSongClassifierBackground(backbone, freeze_base=False, background=cfg.background)

model.load_from_pretrain(cfg.load_name)
#weights = model.state_dict()
#weights.update(torch.load(cfg.load_name))
#weights = torch.load(cfg.load_name)
#model.load_state_dict(weights)
model = model.to(cfg.device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, amsgrad=True, eps=1e-5) # torch.optim.SGD(model.parameters(), lr=1e-3, momentum=5e-4, nesterov=True)#
#optimizer.load_state_dict(torch.load("/kaggle/input/pannsmodel/optimizer-ResNet38-264-birds.h5"))
reducer = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)





In [ ]:
print(model)

In [ ]:
best_score = 0.
best_score5 = 0.

In [ ]:
for e in range(cfg.epochs):
    train_fn(model, train_dataloader, optimizer, loss_fnt, cfg, accumulation=cfg.accumulation, l_mixup=cfg.l_mixup)

    preds, targets, val_loss =inference_fn(model, val_dataloader, optimizer, cfg.device, loss_fnt_test, activation=False, background=cfg.background)
    preds5, targets5, val_loss =inference_fn(model, val_dataloader5, optimizer, cfg.device, loss_fnt_test, activation=False, background=cfg.background)
    #preds = np.argmax(preds, axis=1)
    #targets = np.argmax(targets, axis=1)
    score = f1_micro_row_wise(targets, preds)#np.mean(f1_score_multi(targets, preds))
    score5 = f1_micro_row_wise(targets5, preds5)#np.mean(f1_score_multi(targets5, preds5))
    reducer.step(val_loss)
    if best_score < score:
        best_score = score
        torch.save(model.state_dict(), cfg.save_name)
        torch.save(optimizer.state_dict(), "optimizer-"+ cfg.save_name)

        print("F1 - score improved : ", score)
    else:
        print("F1 score not improve : ", score, " Best : ", best_score)
        
    if best_score5 < score5:
        best_score5 = score5
        torch.save(model.state_dict(), "site1_"+cfg.save_name)

        print("F1 - score improved : ", score5)
    else:
        print("F1 score not improve : ", score5, " Best : ", best_score5)